In [4]:
#import libraries
import mysql.connector
import pandas as pd
from mysql.connector import errors


In [5]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "Boyfaded7552",
    database = "dannys_dinner"
)
my_cursor = mydb.cursor()


What is the total amount each customer spent at the restaurant?


In [6]:
total_amt = pd.read_sql('''WITH total AS (SELECT customer_id, price
FROM sales
INNER JOIN menu
ON sales.product_id = menu.product_id)
SELECT customer_id, SUM(price)
FROM total
GROUP BY customer_id;''', mydb)
total_amt

C:\Users\hp\AppData\Local\Temp\ipykernel_16928\141779315.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_amt = pd.read_sql('''WITH total AS (SELECT customer_id, price


,customer_id,SUM(price)
0,A,76.0
1,B,74.0
2,C,36.0


Customer A spent $ 76
Customer B spent $ 74
Customer C spent $ 36
Customer A was the highest spender ($74)

How many days has each customer visited the restaurant?


In [7]:
days_visited = pd.read_sql('''SELECT sales.customer_id, COUNT(DISTINCT(sales.order_date))
from sales
GROUP BY customer_id;''', mydb)
days_visited

C:\Users\hp\AppData\Local\Temp\ipykernel_16928\1889214176.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  days_visited = pd.read_sql('''SELECT sales.customer_id, COUNT(DISTINCT(sales.order_date))


,customer_id,COUNT(DISTINCT(sales.order_date))
0,A,4
1,B,6
2,C,2


I used distinct to count unique days.
Customer A visited on 4 unique days
Customer B visited on 6 unique days
Customer C visted on 2 unique days.

What was the first item from the menu purchased by each customer?


In [13]:
first_item = pd.read_sql('''WITH first_item AS (SELECT sales.customer_id, sales.order_date, menu.product_name, 
DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date ASC) 'Ranking'
FROM sales
INNER JOIN
menu 
ON sales.product_id = menu.product_id)
SELECT DISTINCT customer_id, product_name
FROM first_item
GROUP BY 1
ORDER BY customer_id;''', mydb)
first_item


C:\Users\hp\AppData\Local\Temp\ipykernel_16928\2163685682.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  first_item = pd.read_sql('''WITH first_item AS (SELECT sales.customer_id, sales.order_date, menu.product_name,


,customer_id,product_name
0,A,sushi
1,B,curry
2,C,ramen


What is the most purchased item on the menu and how many times was it purchased by all customers?


In [14]:
most_purchased = pd.read_sql('''SELECT menu.product_name, COUNT(sales.product_id)
FROM menu
JOIN sales
ON menu.product_id = sales.product_id
GROUP BY menu.product_name
ORDER BY sales.product_id DESC
LIMIT 1;''', mydb)
most_purchased

C:\Users\hp\AppData\Local\Temp\ipykernel_16928\659634045.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  most_purchased = pd.read_sql('''SELECT menu.product_name, COUNT(sales.product_id)


,product_name,COUNT(sales.product_id)
0,ramen,8


Ramen is the most purchased product. It was purchased 8 times

Which item was the most popular for each customer?


In [71]:
most_popular = pd.read_sql('''WITH most_popular AS (SELECT menu.product_name, sales.customer_id, COUNT(sales.product_id) AS quantity ,
DENSE_RANK() OVER(PARTITION BY sales.customer_id ORDER BY COUNT(sales.product_id) DESC ) AS ranks
FROM menu
JOIN
sales
ON menu.product_id = sales.product_id
GROUP BY 1,2)
SELECT product_name, customer_id, quantity
FROM most_popular
WHERE ranks = 1;''', mydb)
most_popular

,product_name,customer_id,quantity
0,ramen,A,3
1,curry,B,2
2,sushi,B,2
3,ramen,B,2
4,ramen,C,3


Which item was purchased first by the customer after they became a member?


In [72]:
first_purchased_as_member = pd.read_sql('''WITH first_item_as_member AS (SELECT sales.customer_id, sales.order_date, menu.product_name, members.join_date,
DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date) AS Ranking
FROM sales 
JOIN members ON sales.customer_id = members.customer_id
JOIN  menu ON sales.product_id = menu.product_id
WHERE order_date >= join_date)
SELECT customer_id, product_name,order_date
FROM first_item_as_member
WHERE Ranking = 1;''', mydb)
first_purchased_as_member

,customer_id,product_name,order_date
0,A,curry,2021-01-07
1,B,sushi,2021-01-11


Which item was purchased just before the customer became a member?


In [73]:
first_item_purchased_before_member = pd.read_sql('''WITH first_item_before_member AS (SELECT sales.customer_id, sales.order_date, menu.product_name, members.join_date,
DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY order_date) AS Ranking
FROM sales 
JOIN members ON sales.customer_id = members.customer_id
JOIN  menu ON sales.product_id = menu.product_id
WHERE order_date <= join_date)
SELECT customer_id, product_name,order_date
FROM first_item_before_member
WHERE Ranking = 1;''', mydb)
first_item_purchased_before_member

,customer_id,product_name,order_date
0,A,sushi,2021-01-01
1,A,curry,2021-01-01
2,B,curry,2021-01-01


What is the total items and amount spent for each member before they became a member?


In [16]:
total_items_amt = pd.read_sql('''SELECT COUNT(menu.product_name), sales.customer_id, SUM(menu.price)
FROM sales
INNER JOIN menu ON menu.product_id = sales.product_id
INNER JOIN members ON members.customer_id = sales.customer_id
WHERE join_date > order_date
GROUP BY customer_id;''', mydb)
total_items_amt

C:\Users\hp\AppData\Local\Temp\ipykernel_16928\3933143686.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_items_amt = pd.read_sql('''SELECT COUNT(menu.product_name), sales.customer_id, SUM(menu.price)


,COUNT(menu.product_name),customer_id,SUM(menu.price)
0,3,B,40.0
1,2,A,25.0


f each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [76]:
total_points = pd.read_sql('''WITH total AS (SELECT sales.customer_id, 
CASE WHEN sales.product_id = 1 THEN price * 20
ELSE menu.price * 10
END AS number_points
from sales
INNER JOIN menu
ON sales.product_id = menu.product_id)
SELECT customer_id, SUM(number_points)
FROM total
GROUP BY customer_id;''', mydb)
total_points

,customer_id,SUM(number_points)
0,A,860.0
1,B,940.0
2,C,360.0


In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

C:\Users\hp\AppData\Local\Temp\ipykernel_16928\1884765530.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  points_by_jan = pd.read_sql('''WITH points_cte AS (SELECT s.customer_id, s.order_date, s.product_id,


DatabaseError: Execution failed on sql 'WITH points_cte AS (SELECT s.customer_id, s.order_date, s.product_id,
			CASE WHEN s.product_id = 1 THEN price * 20
     			     WHEN s.order_date >= mem.join_date AND s.order_date < (mem.join_date + interval '7 days') THEN price * 20
			     ELSE price * 10 END AS points
		    FROM sales s
		    JOIN members mem
			ON s.customer_id = mem.customer_id
		    JOIN menu m
			ON s.product_id = m.product_id 
		    WHERE s.order_date <= to_date('2021 01 31','YYYY MM DD'))
SELECT customer_id, SUM(points) AS points
FROM points_cte
GROUP BY customer_id;': 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ') THEN price * 20
			     ELSE price * 10 END AS points
		    FROM sales s
		   ' at line 3

Recreate the following table output using the available data:


In [77]:
recreate_table = pd.read_sql('''SELECT sales.customer_id, sales.order_date, menu.product_name, menu.price,
CASE WHEN order_date <= join_date THEN 'N' 
ELSE 'Y' 
END AS member
FROM sales
LEFT JOIN menu ON sales.product_id = menu.product_id
LEFT JOIN members ON sales.customer_id = members.customer_id;''', mydb)
recreate_table

,customer_id,order_date,product_name,price,member
0,A,2021-01-01,sushi,10,N
1,A,2021-01-01,curry,15,N
2,A,2021-01-07,curry,15,N
3,A,2021-01-10,ramen,12,Y
4,A,2021-01-11,ramen,12,Y
5,A,2021-01-11,ramen,12,Y
6,B,2021-01-01,curry,15,N
7,B,2021-01-02,curry,15,N
8,B,2021-01-04,sushi,10,N
9,B,2021-01-11,sushi,10,Y


Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.



In [78]:
rank_values = pd.read_sql('''WITH members AS (SELECT sales.customer_id, sales.order_date, menu.product_name, menu.price,
CASE WHEN order_date <= join_date THEN 'N' 
ELSE 'Y' 
END AS member
FROM sales
LEFT JOIN menu ON sales.product_id = menu.product_id
LEFT JOIN members ON sales.customer_id = members.customer_id)
SELECT customer_id, order_date, product_name, price,
CASE WHEN member = 'Y' THEN RANK() OVER (PARTITION BY customer_id,member ORDER BY order_date) ELSE NULL END AS ranking
FROM members;''', mydb)
rank_values

,customer_id,order_date,product_name,price,ranking
0,A,2021-01-01,sushi,10,NaN
1,A,2021-01-01,curry,15,NaN
2,A,2021-01-07,curry,15,NaN
3,A,2021-01-10,ramen,12,1.0
4,A,2021-01-11,ramen,12,2.0
5,A,2021-01-11,ramen,12,2.0
6,B,2021-01-01,curry,15,NaN
7,B,2021-01-02,curry,15,NaN
8,B,2021-01-04,sushi,10,NaN
9,B,2021-01-11,sushi,10,1.0
